In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import mpmath
from fractions import Fraction
from math import floor
import pickle
import sympy as sp
from scipy.optimize import linprog

It's better to use Fractions, as the computation is exact and the difference between two assortments can become smaller and smaller as the stock increases, until it's less than the cumulative effect of float errors.

# MNL and cardinality constraints

Put $k = len(l)$ for having the unconstrained problem.

In [ ]:
# compute E[T_{l1,..,lN}] given the assortment offered at the first step
def T(l, S, list_v, k, memory, assortment):
    """Compute the total expected number of customer required to clear the stock,
    for a policy who offers first the assortment S until a product is selected by a customer, then it follows the optimal policy. 

    Args:
        l (list of integers): the current stock
        S (set): the first assortment offered to the customers, until a product is selected by a customer
        list_v (list): list of the preference weights, without the no-purchase
        k (int): cardinality constraint
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.

    Returns:
        float: the total expected time
    """
    I_N = np.identity(len(list_v)) #identity matrix
    T = Fraction(1,1) #initialization of T
    V_S = Fraction(0,1) #Initialization of V(S)
    for i in S :
        V_S = V_S + list_v[i] # V(S) update
        if tuple(l - I_N[i]) in memory : # if the state l - e_i was already computed, we use this value
            Ti = memory[tuple(l - I_N[i])]
        else : #otherwise, we do the computation
            Ti = f(l - I_N[i],k, list_v, memory, assortment)[0]
        T = T + list_v[i] *Ti # Update of T
    return T/V_S + Fraction(1,1) 

# compute the subsets of cardinality at most k
def generate_subsets_up_to_k(elements, k):
    """Generate a list of the subsets of elements, of cardinality at most k

    Args:
        elements (list): list of the products which are still in stock
        k (int): cardinality constraint

    Returns:
        list: list of the subsets of elements, of cardinality at most k
    """
    subsets = []
    n = len(elements) # number of products which are still in stock
    for i in range(1 << n):  # Nombre de sous-ensembles possibles = 2^n
        subset = [elements[j] for j in range(n) if (i & (1 << j))]
        if len(subset) <= k and len(subset) >= 1:
            subsets.append(subset)
    return subsets

# compute the feasible assortments
def set_assortments(l,k):
    """Return the list of the feasible assortments

    Args:
        l (list of int): current stock
        k (int): cardinality constraint

    Returns:
        list: list of the feasible assortments
    """
    N_assortment = [i for i in range(len(l)) if l[i] > 0] #list of the products which are still in stock
    return generate_subsets_up_to_k(N_assortment, k)

# compute the assortment corresponding to the minimum, and the value of the minimum
def f(l, k, list_v, memory, assortment) :
    """Return the optimal value, after exploring all the feasible solutions.

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    Returns:
        float or Fraction: optimal value
        list : optimal assortment
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    """
    if tuple(l) in memory : # If it was already computed, we return this value
        return memory[tuple(l)], assortment[tuple(l)], memory, assortment
    S_set = set_assortments(l,k) #Compute the list of the feasible assortments
    S_min = []
    T_min = float('inf')
    for S in S_set : #Exploring all the feasible solutions
        T_s = T(l, S, list_v, k, memory, assortment)
        if T_s < T_min: #Updating the optimal assortment and the optimal value
            T_min = T_s
            S_min = S
    assortment[tuple(l)] = S_min #Updating the dictionaries
    memory[tuple(l)] = T_min
    return T_min, S_min, memory, assortment

def solution_opt(l,k, list_v, memory, assortment):
    """Return the dictionaries corresponding to the optimal solution

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    Returns:
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.
    """
    N = len(l)
    if tuple([0 for i in range(N)]) not in memory : #initialization of the dictionaries
        memory[tuple([0 for i in range(N)])] = Fraction(0,1)
        assortment[tuple([0 for i in range(N)])] = []
    if np.sum(l) == 0 :
        return Fraction(0,1),[]
    return f(l,k,list_v, memory, assortment)[2:4]

In [ ]:
memory = {}
assortment = {}
list_v = [Fraction(1,1), Fraction(4,1), Fraction(5,1)]

In [ ]:
# If you modify the preference weights, you need to reset the dictionaries
list_l = [10,10,10]
memory, assortment = solution_opt(list_l,2,list_v, memory, assortment)

In [ ]:
memory[1,1,1]

Fraction(737, 180)